<!-- Auto table of contents -->
<h1 class='tocIgnore'>Naturalist : Extraction et exploitation des traces</h1>
<ul>
  <li>présentes dans les formulaires à partir Naturalist V0.128 (ou beta mai 2019),</li>
  <li>à condition de cocher la case "Enregistrer ma trace" en début de formulaire,</li>
  <li>via l'export Excel exclusivement
      (pas encore d'API pour ça, et absent des exports XML, JSON, KML, CSV en décembre 2019),</li>
  <li>avec la colonne "trace" sélectionnée dans l'export,</li>
  <li>uniquement via Faune-France (pas dispo. via les sites régionaux).</li>
</ul>
<p>Lecture XLSX et carto. publiée avec données anonymisées sur https://framagit.org/lpo/partage-de-codes le 25/01/2020</p>
<div style="overflow-y: auto">
  <h2 class='tocIgnore'>Table des matières</h2>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('ipython_notebook_toc.js')

In [ ]:
import sys
import os
import pathlib as pl
import datetime as dt
from lxml import etree

import pandas as pd
import numpy as np

import folium
import folium.plugins

import pyproj
from shapely import geometry

import matplotlib.pyplot as plt

from collections import OrderedDict as odict
import json

from IPython.display import HTML

In [ ]:
import autods as ads
import visionat as vsn

In [ ]:
# Configure logging.
import logging

# Setup given logger.
def setupLogger(logr, level=logging.ERROR, handlers=[sys.stdout], fileMode='w',
                format='%(asctime)s %(name)s %(levelname)s\t%(message)s'):
    
    # Cleanup any default handler (jupyter does some logging initialisation itself ...)
    while logr.handlers:
        logr.removeHandler(logr.handlers[-1])

    # Set new handlers
    formatter = logging.Formatter(format)
    
    # Set level
    logr.setLevel(level)
    
    # Setup new handlers
    for hdlr in handlers:
        if isinstance(hdlr, str):
            handler = logging.FileHandler(hdlr, mode=fileMode)
        else:
            handler = logging.StreamHandler(stream=hdlr)
        handler.setFormatter(formatter)
        logr.addHandler(handler)
    
    def handlerId(hdlr):
        return 'File({})'.format(hdlr) if isinstance(hdlr, str) else 'Stream({})'.format(hdlr.name)
    logr.info('Logging to {}.'.format(', '.join(handlerId(hdlr) for hdlr in handlers)))

    return logr

# Local (NB) logger
setupLogger(logging.getLogger('autods'), level=logging.INFO,
            handlers=[sys.stdout, 'tmp/natrans.log'])
setupLogger(logging.getLogger('visionat'), level=logging.INFO,
            handlers=[sys.stdout, 'tmp/natrans.log'])
logger = setupLogger(logging.getLogger('natrans'), level=logging.DEBUG,
                     handlers=[sys.stdout, 'tmp/natrans.log'])

# I. Chargement des données (issues d'exports de Faune-France)

## 1. Paramètres d'import / filtrage (fichier, observateur, commentaire liste, ...)

In [ ]:
tracesManu = False

In [ ]:
# ACDC 2019 JPM (en fait, trace enregistrée uniquement sur le 2nd passage, et pour 15 points sur 17)
ficSrcVN = 'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602ff.xlsx'
obser, obserAbbv = 'Jean-Philippe Meuret', 'JPM'

In [ ]:
# Formulaires Romain avec trace du 25/05/2019 au 25/01/2020 (tous)
# * Impluvium: "epoc" & Saint-Ours, Pulvérières, Charbonnière-les-Varennes
# * et le reste : des EPOC et des transects
ficSrcVN = 'tmp/FormulairesRR-traces-20190525-20200125ff.xlsx'
comntRE = 'ACDC'
obser, obserAbbv = 'Romain Riols', 'RR'

In [ ]:
# ACDC 2019 Romain 25/05/2019
ficSrcVN = 'tmp/ACDC2019-Naturalist-FormulairesRR20190425ff.xlsx'
comntRE = 'ACDC'
obser, obserAbbv = 'Romain Riols', 'RR'

In [ ]:
# Test JPM jardin 23/05/2019
#ficSrcVN = f'tmp/NaturalistTestTrace-JardinJPM20190523ff.xlsx'
#comntRegexp = '???'
#obser, obserAbbv = 'Jean-Philippe Meuret', 'JPM'

In [ ]:
# Transects Vergers de Tallende Cyrille printemps 2020
ficSrcVN = 'transects/ExportTransects2020T2-CJS-FF.xlsx'
comntRE = 'verger.*tallende'
obser, obserAbbv = 'Cyrille Jallageas', 'CJS'
tracesManu = True # False pour utiliser les traces Naturalist (bruitées !)
nomEtude = 'TallendeVergers2020'
dZoneEtude = dict(Zone='Vergers de Tallende', Surface=150) # ha
groupage = False # Clustering analyses DS

## 2. Chargement

In [ ]:
obsCols = ['ID liste', 'Liste complète ?', 'Commentaire de la liste',
           'Date', 'Ref', 'Horaire', 'Lieu-dit', 'Commune', 'Nom scientifique',
           'Estimation', 'Nombre', 'Détails', 'Code atlas',
           'Lat (WGS84)', 'Lon (WGS84)', 'UTM X [m]', 'UTM Y [m]', 'Remarque', 'Trace']

In [ ]:
# Fonctions pour certaines colonnes calculées.
def nomTransect(sSight):
    
    listCmnt = sSight['Commentaire de la liste']
    if pd.isnull(listCmnt):
        return None
    
    if listCmnt.lower().find('transect s') >= 0:
        return 'Sud2'  # Attention: Il y aura un Sud1 "fait à la main" (voir ci-dessous).
    
    return 'Nord'

In [ ]:
sightCountsParser = vsn.SightingCountsParser(atlasCodex='EBCC',  # Le codex des codes Atlas est l'EBCC sur FF !
                                             categoryCols=['nMalAd', 'nAutAd', 'nJuv', 'nVol'])

In [ ]:
vnds = vsn.VisionatureDataSet(ficSrcVN, keepCols=obsCols, dCompCols=dict(Transect=nomTransect), 
                              listExtraCols=['Transect'], 
                              listsHaveTrace=True, listTraceLengthCol='Longueur',
                              sightCountsParser=sightCountsParser)

In [ ]:
vnds.columns

In [ ]:
vnds.dfData.head()

In [ ]:
vnds.dfData['Commentaire de la liste'].nunique(), vnds.dfData['Commentaire de la liste'].unique()

# II. Filtrage des données

* formulaires
* avec commentaire ad-hoc (si spécifié via comntRE)

In [ ]:
# Uniquement les données des formulaires
vnds.cleanup(nonLists=True)
#vnds.dropRows(vnds.dfData['ID liste'] == 0)
len(vnds)

In [ ]:
# Uniquement les données respectant le critère de commentaire liste
if comntRE:
    vnds.dropRows(vnds.dfData['Commentaire de la liste'].isnull())
    vnds.dropRows(~vnds.dfData['Commentaire de la liste'].str.contains(comntRE, case=False))
len(vnds)

In [ ]:
# Uniquement les formulaires avec trace si on ne les a pas par ailleurs
if not tracesManu:
    vnds.cleanup(emptyTraces=True)

len(vnds)

# III. Autres filtrages / traitements spécifiques des données

## 1. Transects de Cyrille dans les vergers de Tallende au printemps 2020

* séparation du 1er transect (28/03) en 2 transects : le Nord (comme les autres jours),
  et un Sud2 jamais refait (étape 1 : suppression des données ; il restera à nettoyer la trace, Cf. ci-dessous)

In [ ]:
# Suppression des données entre le demi-transect  Nord et le demi-Sud2 (1er tronçons ~N-S)
vnds.dropRows((vnds.dfData['Date'] == '2020-03-28')
              & (vnds.dfData['Horaire'] > '09:37') & (vnds.dfData['Horaire'] < '09:45'))

# Suppression des données entre le demi-transect Sud2 et retour au point de dé part (2nd tronçons ~N-S)
vnds.dropRows((vnds.dfData['Date'] == '2020-03-28') & (vnds.dfData['Horaire'] > '10:13'))

# Création d'une nouvelle liste pour le demi-tracsect Sud2
idNouvListe = vnds.dfData.loc[vnds.dfData['Date'] == '2020-03-28', 'ID liste'].iloc[0] + 1
assert idNouvListe not in vnds.dfData['ID liste']

vnds.dfData.loc[(vnds.dfData['Date'] == '2020-03-28')
                & (vnds.dfData['Horaire'] >= '09:45') & (vnds.dfData['Horaire'] <= '10:13'),
                ['ID liste', 'Transect']] = idNouvListe, 'Sud1'

len(vnds)

In [ ]:
vnds.dfData[['Transect', 'ID liste', 'Date', 'Commentaire de la liste', 'Trace']] \
    .drop_duplicates(['Date', 'ID liste'])

# IV. Examen des données

In [ ]:
dfObs = vnds.dfData

In [ ]:
# Toutes des listes complètes ?
dfObs[dfObs['Liste complète ?'] != 1]

In [ ]:
# Vérifier que la trace de chaque liste est présente à l'identique dans toutes les données de la liste
dfObs[['ID liste', 'Date', 'Trace']].groupby(['ID liste', 'Date']).nunique().Trace.value_counts()

In [ ]:
# Nb de traces identique par liste
dfObs[['ID liste', 'Date', 'Trace']].groupby(['ID liste', 'Date']).nunique().Trace.reset_index()

In [ ]:
# Les formulaires à traiter
dfObs[['ID liste', 'Liste complète ?', 'Commentaire de la liste']].drop_duplicates()

In [ ]:
# Nbres de données par liste
dfObs.groupby(['Transect', 'ID liste', 'Date']).count()[['Nombre']]

In [ ]:
# Nbres de données moyen par transect type
dfObs.groupby(['Transect', 'ID liste', 'Date']).count()[['Nombre']] \
     .reset_index().groupby('Transect').mean()[['Nombre']]

# V. Extraction / décodage des traces GPS

## 1. A partir des données de terrain (colonne Trace)

(mais ... attention à la qualité ... imprécision GPS !)

### a. Extraction / décodage

In [ ]:
if not tracesManu:
    
    dfTracesGps = vnds.listTraces()
    
    print(dfTracesGps)

### b. Correction spécifiques après décodage

#### i. Transects de Cyrille dans les vergers de Tallende au printemps 2020

Après III.1, les 2 1ères listes ont la même trace trop longue correspondant à la boucle du 28/03 ;
il faut donc les néttoyer pour obtenir : liste 876746 = transet Nord, liste 876747 = transect Sud1.

In [ ]:
if not tracesManu:
    
    print('avant :', len(dfTracesGps))
    
    # Nettoyage de la trace de la 876746 => transect Nord
    sLabels2Drop = dfTracesGps[(dfTracesGps['ID liste'] == 876746) & ~(dfTracesGps.ptIndx.between(8, 170))].index
    dfTracesGps.drop(sLabels2Drop, inplace=True)
    
    # Nettoyage de la trace de la 876747 => transect Sud1
    sLabels2Drop = dfTracesGps[(dfTracesGps['ID liste'] == 876747) & ~(dfTracesGps.ptIndx.between(199, 325))].index
    dfTracesGps.drop(sLabels2Drop, inplace=True)

    print('après :', len(dfTracesGps))

In [ ]:
if not tracesManu:
    
    print(dfTracesGps[dfTracesGps['ID liste'] == 876746])
    print(dfTracesGps[dfTracesGps['ID liste'] == 876747])

### c. Remplacement des traces dans le jeu de données

In [ ]:
if not tracesManu:
    
    # Ecrasement des traces du jeu de données
    vnds.setListTraces(dfTracesGps)

In [ ]:
if not tracesManu:
    
    dfTracesGps = vnds.listTraces()
    print(len(dfTracesGps))

In [ ]:
if not tracesManu:
    
    print(dfTracesGps[dfTracesGps['ID liste'] == 876746])
    print(dfTracesGps[dfTracesGps['ID liste'] == 876747])

## 2. A partir de traces précises obtenues par ailleurs

* tracés précisément à la main via GeoPortail,
* pas de pb avec l'imprécision crasse du GPS,
* en remplacement des traces relevées ...

In [ ]:
vnds.lists().drop(columns='Trace')

### a. Transects de Cyrille dans les vergers de Tallende au printemps 2020

In [ ]:
if tracesManu:
    
    dTraces = { sList['ID liste']: f'transects/Transect{sList.Transect}-VergersTallende-CJS-2020T2.kml' \
                for _, sList in vnds.lists().iterrows() }

    vnds.setListTraces(dTraces)
    
    print(vnds.listTraces().head(10))
    
else:
    
    print('On va utiliser les traces Naturalist')

# VI. (optionnel) Cartographie des données et traces des formulaires

In [ ]:
# Serveurs et couches carto. pour folium / Leaflet
mdOSM = dict(tiles='http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
             attr='Open Street Map',
             name='Open Street Map', max_zoom=22, photo=False)

mdOTM = dict(tiles='http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
             attr='<a href="https://opentopomap.org/">OpenTopoMap</a> '
                  '(<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)',
             name='Open Topo Map', max_zoom=22, photo=False)
mdThOut = dict(tiles='https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png',
               attr='Thunderforest Outdoors', 
               name='Thunderforest Outdoors', max_zoom=22, photo=False)

mdSatArcGis = dict(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                   attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid,'
                        ' IGN, IGP, UPR-EGP, and the GIS User Community',
                   name='ArcGIS Satellite', max_zoom=22, photo=True)

def sight2String(sSight):
    
    ref = sSight['Ref'] if 'Ref' in sSight.index else ''
    
    esp = ''.join(w[:4].title() for w in sSight['Nom scientifique'].split())
    
    if 'nMalAd' in sSight.index:
        nbre = ', '.join('{}={}'.format(col, int(sSight[col])) \
                         for col in ['nMalAd', 'nAutAd', 'nJuv', 'nVol'] \
                         if col in sSight.index and not pd.isnull(sSight[col]) and sSight[col] > 0)
    else:
        nbre = str(int(sSight.Nombre))
        if not pd.isnull(sSight['Détails']):
            nbre += '[{}]'.format(sSight['Détails'])
        
    dist = ('d='+str(int(sSight['Distance']))) if 'Distance' in sSight.index else ''
    codAtlas = 0 if pd.isnull(sSight['Code atlas']) else int(sSight['Code atlas'])
                                
    return '#{} {} {} {} {} (code {}) {}'.format(ref, sSight.Date.date().isoformat(), sSight.Horaire,
                                                 esp, nbre, codAtlas, dist)
    
# Colors for sightings and traces, wether the map layer is a photo (dark) or not (clear).
DColors = { True: dict(sight='greenyellow', trace=dict(point='magenta', segment='cyan')), 
            False: dict(sight='green', trace=dict(point='red', segment='mediumblue')) }

def color(mapSrc, sightPoint=False, tracePoint=False, traceSegment=False):
    isMapDark = mapSrc['photo']
    dColors = DColors[isMapDark]
    if sightPoint:
        clr = dColors['sight']
    elif tracePoint:
        clr = dColors['trace']['point']
    elif traceSegment:
        clr = dColors['trace']['segment']
    else:
        raise Exception('No target selected for color')
        
    return clr
    
def showOnMap(dfTrace, dfSights, trTitle='', mapSrc=mdOSM, sight2String=sight2String):
    
    mp = folium.Map(**mapSrc)

    # La trace
    # a. Les lignes reliant les points
    dfTrace = dfTrace.append(dfTrace.iloc[-1]) # Duplicate last points to keep all after shift below
    dfTrace['lon_sfd'] = dfTrace.lon.shift(-1)
    dfTrace['lat_sfd'] = dfTrace.lat.shift(-1)
    dfTrace.dropna(inplace=True)

    lines = list(zip(zip(dfTrace.lat, dfTrace.lon), zip(dfTrace.lat_sfd, dfTrace.lon_sfd)))[:-1]
    pline = folium.PolyLine(lines, color=color(mapSrc, traceSegment=True),
                            weight=1, opacity=0.6, popup=folium.Popup(trTitle))
    pline.add_to(mp)

    # b. Les points
    for _, sPt in dfTrace.iterrows():
        mrk = folium.CircleMarker(location=(sPt.lat, sPt.lon), 
                                  popup=folium.Popup('#{}: {}'.format(sPt['ID liste'], sPt.ptIndx)),
                                  color=color(mapSrc, tracePoint=True),
                                  radius=2, weight=2, fill=True)
        mrk.add_to(mp)

    # Les données
    for indSight, sSight in dfSights.iterrows():
        mrk = folium.CircleMarker(location=(sSight.lat, sSight.lon), radius=3, 
                                  color=color(mapSrc, sightPoint=True), fill=True,
                                  popup=folium.Popup('#{}: {}'.format(indSight, sight2String(sSight))))
        mrk.add_to(mp)

    mp.fit_bounds(mp.get_bounds())
    
    return mp

In [ ]:
print(', '.join(str(date.date()) for date in vnds.dfData.Date.drop_duplicates()))

In [ ]:
# Les listes
vnds.lists().drop(columns='Trace')

## 1. Sélection des listes à tracer en même temps

In [ ]:
vnds.lists().groupby('Date').apply(lambda df: df['ID liste'].to_list())

In [ ]:
# Soit en dur.
dfListes = vnds.lists(ids=[876746]) #[955938, 956038] #[880581, 880582] #[876746]

In [ ]:
# Soit: Iteration sur les listes du jeu de données, groupées par dates
itListes = iter(vnds.lists().groupby('Date'))

In [ ]:
dfListes = next(itListes)[1]
dfListes

## 2. Extraction des points des traces du jour et des données

In [ ]:
idsListes = dfListes['ID liste'].values

dfTracesJour = vnds.listTraces(listIds=idsListes)

dfObsJour = vnds.sightings(listIds=idsListes)

dict(listes=idsListes, obs=len(dfObsJour), traces=len(dfObsJour))

In [ ]:
dfObsJour[['Lon (WGS84)', 'Lat (WGS84)', 'Date', 'Horaire',
           'Nom scientifique', 'Détails', 'Nombre', 'Code atlas']]

## 3. Tracé

In [ ]:
showOnMap(dfTracesJour,
          dfObsJour.rename(columns={'Lon (WGS84)': 'lon', 'Lat (WGS84)': 'lat' }), mapSrc=mdOSM,
          trTitle='#{} {}'.format(','.format(list(idsListes)), list(vnds.lists(ids=idsListes)['Transect'])))

In [ ]:
dfListes.drop(columns='Trace')

# VII. Décodage des effectifs comptés

In [ ]:
vnds.computeSightingCounts()

vnds.columns

In [ ]:
vnds.dfData[['ID liste', 'Horaire', 'Date', 'Nom scientifique',
             'Nombre', 'Détails', 'Code atlas', 'nMalAd', 'nAutAd', 'nJuv', 'nVol']].head(30)

In [ ]:
vnds.dfData[['nMalAd', 'nAutAd', 'nJuv', 'nVol']].sum().sum(), vnds.dfData['Nombre'].sum()

In [ ]:
vnds.toExcel('tmp/after.xlsx') # => Vérifier à l'oeil les effectifs de détail

In [ ]:
vnds.dfData['Détails'].fillna('').str.contains('vol')

In [ ]:
vnds.dfData.loc[vnds.dfData['Détails'].fillna('').str.contains('vol'),
                ['ID liste', 'Horaire', 'Date', 'Nom scientifique',
                 'Nombre', 'Détails', 'Code atlas', 'nMalAd', 'nAutAd', 'nJuv', 'nVol']]

# VIII. Calcul des distances observateur - oiseau

(la distance minimale entre l'oiseau et la poly-ligne du transect)

In [ ]:
vnds.computeTraceSightDistances(distanceCol='Distance')

vnds.dfData[['ID liste', 'Horaire', 'Date', 'Nom scientifique',
             'Nombre', 'Détails', 'Code atlas', 'nMalAd', 'nAutAd', 'nJuv', 'nVol', 'Distance']].head(20)

In [ ]:
np.histogram(vnds.dfData.Distance, bins=10)[0]

In [ ]:
vnds.dfData.Distance.hist(figsize=(16, 4), bins=40)

# IX. Extraction des inventaires

In [ ]:
# Les inventaires (les transects)
dfInventaires = vnds.lists(columns=['ID liste', 'Date', 'Transect', 'Longueur']).copy()
dfInventaires.Longueur = dfInventaires.Longueur.astype(int)
dfInventaires

# X. Individualisation des données

In [ ]:
vnds.columns

In [ ]:
# Colonnes d'effectifs à prendre en compte (on ignore les autres)
countCols =  ['nMalAd', 'nAutAd']

# Calcul des catégories : 1 seule, "Adulte" = Mâle ou Autre.
def count2AdultCat(sCounts):
    return 'm' if 'Mal' in sCounts[sCounts > 0].index[0] else 'a'

# Création d'un FieldDataSet
fds = ads.FieldDataSet(vnds.sightings(), countCols=countCols, addMonoCatCols={ 'Adulte': count2AdultCat })

# ... pour individualiser et catégoriser les données.
dfObsCatIndiv = fds.individualise()
print(dfObsCatIndiv[countCols].sum().to_dict(), 'individus')

# On ne garde que les colonnes utiles (comptes à 0 ou 1 <=> catégories), et avec des noms améliorés
dfObsCatIndiv = dfObsCatIndiv[['ID liste', 'Date', 'Transect',
                               'Horaire', 'Nom scientifique', 'Distance', 'Adulte']].copy()
dfObsCatIndiv.rename(columns={ 'Nom scientifique': 'Espèce' }, inplace=True)

print(len(dfObsCatIndiv), 'individus au total')

In [ ]:
# On ne garde que les colonnes utiles (comptes à 0 ou 1 <=> catégories), et avec des noms améliorés
dfObsCatIndiv = dfObsCatIndiv[['ID liste', 'Date', 'Transect',
                               'Horaire', 'Espèce', 'Distance', 'Adulte']].copy()
len(dfObsCatIndiv)

In [ ]:
dfObsCatIndiv.head()

In [ ]:
# Export des données et inventaires.
nomFicCible = pl.Path('transects') / f'{nomEtude}-ObsIndiv.xlsx'

with pd.ExcelWriter(nomFicCible) as xlsWriter:
    
    dfObsCatIndiv.to_excel(xlsWriter, index=False, sheet_name='Donnees')
    dfInventaires.to_excel(xlsWriter, index=False, sheet_name='Inventaires')
    
print(nomFicCible)

# X. Export pour analyses dans Distance

## 1. Sélection et définition des échantillons

In [ ]:
especes = 20

In [ ]:
# Examen des données : Nombre d'individus par espèce, pour voir quelles espèces on va analyser
if groupage: # Clustering lors des analyses DS
    dfIndivCounts = dfObsCatIndiv[['Espèce', 'Nombre']].groupby('Espèce').sum()
    dfIndivCounts.rename(columns=dict(Nombre='Individus'), inplace=True)
else:
    dfIndivCounts = dfObsCatIndiv[['Espèce', 'Distance']].groupby('Espèce').count()
    dfIndivCounts.rename(columns=dict(Distance='Individus'), inplace=True)

dfIndivCounts.sort_values(by='Individus', ascending=False, inplace=True)

dfIndivCounts[dfIndivCounts.Individus >= especes if isinstance(especes, int) else 20]

In [ ]:
# Spec. implicite des variantes (=> combinaisons à générer automatiquement)
# a. 1 variante espèce ... par espèce <8-]
assert isinstance(especes, list) or isinstance(especes, int)
if isinstance(especes, list):
    varEspeces = especes
else:
    varEspeces = list(dfIndivCounts[dfIndivCounts.Individus >= especes].index)

# b. Variantes adultes.
varAdultes = ['m', 'm+a'] # Tous les adultes ensemble => 1 variante

# c. Variantes passages (= dates, car pas plus d'1 passage par jour).
varPassages = [''] # Tous les passages ensemble => 1 variante

# c. La spec. des variantes
dImplSamples = { 'Espèce': varEspeces, 'Adulte': varAdultes, 'Date': varPassages }
dImplSamples

In [ ]:
# Explicitation des specs.
dfExplSampleSpecs = ads.DSAnalyser.explicitVariantSpecs(odict([('echant_impl', dImplSamples)]))
dfExplSampleSpecs

In [ ]:
dfObsCatIndiv.head()

In [ ]:
# Jeu de données individualisées.
mds = ads.MonoCategoryDataSet(dfObsCatIndiv, dfTransects=dfInventaires, dSurveyArea=dZoneEtude,
                              transectPlaceCols=['Transect'], passIdCol='Date', effortCol='Longueur',
                              sampleDecFields=['Distance'])

In [ ]:
workDir = pl.Path('transects') / (nomEtude + '-dst')

In [ ]:
# Chaîne courte d'identification d'une spec. d'échantillon.
def sampleAbbreviation(sSample):
    
    abrvSpe = ''.join(word[:4].title() for word in sSample['Espèce'].split(' ')[:2])
    
    sampAbbrev = '{}-{}'.format(abrvSpe, sSample.Adulte.replace('+', ''))
    
    return sampAbbrev

In [ ]:
logger.info('Export au format Distance : C\'est parti ...')

# Moteur MCDS pour l'export.
mcds = ads.MCDSEngine(workDir=workDir,
                      distanceUnit='Meter', areaUnit='Hectare',
                      surveyType='Line', distanceType='Perpendicular', clustering=groupage)

# Pour chaque échantillon :
for sampInd, sSamp in dfExplSampleSpecs.iterrows():
    
    sampAbbrev = sampleAbbreviation(sSamp)

    # Selection des données
    sds = mds.sampleDataSet(sSamp)
    if not sds:
        logger.info('#{:02d} : {} => Pas de données, pas de fichier'.format(sampInd+1, sampAbbrev))
        continue

    # Export au format Distance
    fpn = workDir / f'{nomEtude}-{sampAbbrev}-dist.txt'
    fpn = mcds.buildDistanceDataFile(sds, tgtFilePathName=fpn)

    logger.info('#{:02d} : {} => {}'.format(sampInd+1, sampAbbrev, fpn.name))

# Arrêt moteur.
mcds.shutdown()

# Terminé.
logger.info('Terminé.')

# A. Essais de simplification / nettoyage / ... des traces

In [ ]:
dfTracesForms = vnds.listTraces()
len(dfTracesForms)

## 1. Simplification via algo. de Ramer-Douglas-Peucker

=> bof.

In [ ]:
import rdp_fw as rdp # Optimised Ramer-Douglas-Peucker algorithm (by https://github.com/FlorianWilhelm)

In [ ]:
dfCoordsAvant = dfTracesForms[['obseur_lat', 'obseur_lon']]

ax = dfCoordsAvant.plot(x='obseur_lon', y='obseur_lat', figsize=(16, 8))
_ = dfCoordsAvant.plot(ax=ax, x='obseur_lon', y='obseur_lat', style='ro')

In [ ]:
dfCoordsApres = pd.DataFrame(data=rdp.rdp(dfCoordsAvant.values, 0.0001), columns=['obseur_lat', 'obseur_lon'])

ax = dfCoordsApres.plot(x='obseur_lon', y='obseur_lat', figsize=(16, 8))
_ = dfCoordsApres.plot(ax=ax, x='obseur_lon', y='obseur_lat', style='ro')

## 2. Lissage

## a. Modélisation par un polynome, par les moindres carrés

https://mmas.github.io/least-squares-fitting-numpy-scipy

=> bof.

In [ ]:
from scipy import optimize

In [ ]:
dfCoordsAvant = dfTracesForms[['obseur_lat', 'obseur_lon']]

ax = dfCoordsAvant.plot(x='obseur_lon', y='obseur_lat', figsize=(16, 8))
_ = dfCoordsAvant.plot(ax=ax, x='obseur_lon', y='obseur_lat', style='ro')

In [ ]:
# Polynôme de degré N = len(pp)
def polyN(x, *pp):
    y = 0
    for n, p in enumerate(pp):
        y += p*x**(len(pp) - n)
    return y

# Erreur entre donnée (x, y) et son estimée via polyN
def residual(pp, x, y):
    return y - polyN(x, *pp)

In [ ]:
x = dfCoordsAvant['obseur_lon'].values
y = dfCoordsAvant['obseur_lat'].values

In [ ]:
n = 10
p0 = [1.]*n
popt, pcov = optimize.leastsq(residual, p0, args=(x, y))

popt

In [ ]:
xn = np.linspace(min(x), max(x), 10)
yn = polyN(xn, *popt)

plt.figure(figsize=(16, 8))
plt.plot(x, y, 'or')
plt.plot(xn, yn)
plt.show()

### b. Filtrage de Kalman simplifié

Pb: inutilisable, on n'a pas les timestamps des points GPS ...

NON testé !

In [ ]:
# Kalman filter processing for lattitude and longitude
# https:#stackoverflow.com/questions/1134579/smooth-gps-data/15657798#15657798
class GPSKalmanFilter(object):

    def __init__(self, Q_metres_per_second):
    
        self.Q_metres_per_second = Q_metres_per_second

        self.MinAccuracy = 1.
    
        self.TimeStamp_milliseconds = 0
        self.lat = 0.
        self.lng = 0.
            
        # P matrix.  Negative means object uninitialised.
        # NB: units irrelevant, as long as same units used throughout
        self.variance = -1.

    def getTimeStamp(self):
        return self.TimeStamp_milliseconds
    
    def getLat(self):
        return self.lat
    
    def getLng(self):
        return self.lng
    
    def getAccuracy(self):
        return Math.sqrt(self.variance)

    def setState(lat, lng, accuracy, TimeStamp_milliseconds):
        self.lat = lat
        self.lng = lng
        self.variance = accuracy * accuracy
        self.TimeStamp_milliseconds = TimeStamp_milliseconds

    def process(lat_measurement, lng_measurement, accuracy, TimeStamp_milliseconds):
        
        """Kalman filter processing for lattitude and longitude
        :param nlat_measurement_degrees: new measurement of lattidude
        :param lng_measurement: new measurement of longitude
        :param accuracy: measurement of 1 standard deviation error in metres
        :param TimeStamp_milliseconds: time of measurement
        :returns: new state
        """

        if accuracy < self.MinAccuracy:
            accuracy = self.MinAccuracy

        if self.variance < 0:
            # if variance < 0, object is uninitialised, so initialise with current values
            self.SetState(lat_measurement, lng_measurement, accuracy*accuracy, TimeStamp_milliseconds)
        else:
            # else apply Kalman filter methodology
            TimeInc_milliseconds = TimeStamp_milliseconds - self.TimeStamp_milliseconds

            if TimeInc_milliseconds > 0:
                
                # time has moved on, so the uncertainty in the current position increases
                self.variance += TimeInc_milliseconds * self.Q_metres_per_second * self.Q_metres_per_second / 1000
                self.TimeStamp_milliseconds = TimeStamp_milliseconds
                # TO DO: USE VELOCITY INFORMATION HERE TO GET A BETTER ESTIMATE OF CURRENT POSITION

            # Kalman gain matrix K = Covarariance * Inverse(Covariance + MeasurementVariance)
            # NB: because K is dimensionless, it doesn't matter that variance has different units to lat and lng
            K = self.variance / (self.variance + accuracy * accuracy)
            
            # apply K
            self.lat += K * (lat_measurement - self.lat)
            self.lng += K * (lng_measurement - self.lng)
            
            # new Covarariance  matrix is (IdentityMatrix - K) * Covarariance
            self.variance = (1 - K) * self.variance
        
def kalmanFilter(coords, Q_metres_per_second):
    
    kf = GPSKalmanFilter(Q_metres_per_second)
    updatedCoords = []

    for index in range(len(coords)):
        lat, lng, accuracy, timestampInMs = coords[index]
        updatedCoords[index] = kalmanFilter.process(lat, lng, accuracy, timestampInMs)
        
    return updatedCoords

### c. Filtrage de Savitzky-Golay

https://plotly.com/python/smoothing/

C'est mieux, mais il reste du travail ...

In [ ]:
from scipy import signal

In [ ]:
# Avant
ax = dfTracesForms.plot(x='obseur_lon', y='obseur_lat', figsize=(16, 8))
_ = dfTracesForms.plot(ax=ax, x='obseur_lon', y='obseur_lat', style='ro')

In [ ]:
# Après
winLen = 15 # Must be odd
polyOrder = 2

dfTracesForms['obseur_lon_sg'] = \
    signal.savgol_filter(dfTracesForms['obseur_lon'].values, window_length=winLen, polyorder=polyOrder)
dfTracesForms['obseur_lat_sg'] = \
    signal.savgol_filter(dfTracesForms['obseur_lat'].values, window_length=winLen, polyorder=polyOrder)

In [ ]:
# Après
ax = dfTracesForms.plot(x='obseur_lon_sg', y='obseur_lat_sg', figsize=(12, 6))
_ = dfTracesForms.plot(ax=ax, x='obseur_lon_sg', y='obseur_lat_sg', style='ro')

In [ ]:
showOnMap(dfTracesForms.rename(columns={'ID liste': 'formId', 'NumPt': 'ptNum',
                                      'obseur_lon_sg': 'lon', 'obseur_lat_sg': 'lat'}),
          dfObsListe.rename(columns={'Lon (WGS84)': 'lon', 'Lat (WGS84)': 'lat' }), mapSrc=mdOTM,
          trTitle='#{} {}'.format(','.format(idsListe), dfTracesForms.iloc[0]['Commentaire de la liste']))

# B. Comparaison traces / géolocs "réelles"

* entre mes formulaires ACDC 2019 2nd passage, quasi-tous "avec trace",
* et les géolocs "de mémoire" (novembre 2019).

In [ ]:
# Lecture des données brutes avec géolocs de mémoire (produites via NB ACDC-donnees-naturalist IV.4)
dfObsBrutes = pd.read_excel('ACDC/ACDC2019-Naturalist-ObsBrutesAvecDist.xlsx')
dfObsBrutes.drop(dfObsBrutes[dfObsBrutes.Observateur != obser].index, inplace=True)
dfObsBrutes[['Num point ACDC', 'Passage', 'Date', 'Heure début', 'lon_mem', 'lat_mem']]

In [ ]:
dfObsBrutes['Num point ACDC'].unique()

In [ ]:
# Extraction des géolocs de mémoire
dfGeolocMem = dfObsBrutes[['Num point ACDC', 'Passage', 'Date', 'Heure début', 'lon_mem', 'lat_mem']] \
                .groupby(['Num point ACDC', 'Passage']).first()
dfGeolocMem

In [ ]:
# Traces : Ajouts infos Num point ACDC et Passage
# (puisque ID liste spécifiques à Faune XX, différent de ceux de Faune France)
dfTraces['Num point ACDC'] = dfTraces['Commentaire de la liste'].apply(lambda s: int(s.split(' ')[1]))
dfTraces['Passage'] = dfTraces.Date.apply(lambda ts: 'a' if ts < pd.Timestamp('2019-05-15') else 'b')
dfTraces

In [ ]:
# Jointure avec les traces (en gardant tous les points et passages effectués, pas seulement ceux avec trace)
dfTraces = dfTraces.join(dfGeolocMem[['lon_mem', 'lat_mem']], on=['Num point ACDC', 'Passage'], how='right')
dfTraces.reset_index(inplace=True)
dfTraces

In [ ]:
# Suppression des traces du passage a (non dispo. à l'époque)
dfTraces.drop(dfTraces[dfTraces.Passage == 'a'].index, inplace=True)
dfTraces

In [ ]:
dfTraces['Num point ACDC'].unique(), dfTraces['Passage'].unique()

In [ ]:
# Cartographie des formulaires
mp = folium.Map()
folium.TileLayer(**mdSatArcGis).add_to(mp)

# Le contrôle pour changer de couche
folium.LayerControl().add_to(mp)

# Pour chaque point ...
for numPoint in sorted(dfTraces['Num point ACDC'].unique()):
    
    # Les données
    dfObsListe = dfObsBrutes[dfObsBrutes['Num point ACDC'] == numPoint]
    for indObs, sObs in dfObsListe.iterrows():
        mrk = folium.CircleMarker(location=(sObs.lat, sObs.lon), 
                                  radius=2, color='cyan', fill=True,
                                  popup=folium.Popup('#{} {} {} {} {} (code {})' \
                                                     .format(indObs, sObs.Date, sObs.Horaire,
                                                             sObs.Nombre, sObs['Nom latin'], sObs['Code atlas'])))
        mrk.add_to(mp)
    
    # La position observateur "de mémoire", avec cerles concentriques r=10m + STOC EPS pour l'échelle
    dfTracesForms = dfTraces[dfTraces['Num point ACDC'] == numPoint].copy()
    
    latMem, lonMem = dfTracesForms.iloc[0][['lat_mem', 'lon_mem']]
    mrk = folium.CircleMarker(location=(latMem, lonMem), 
                              radius=6, color='orange', fill=True,
                              popup=folium.Popup('#{} Géoloc. de mémoire'.format(numPoint)))
    mrk.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=10, color='orange', weight=1,
                        popup=folium.Popup('Rayon 10m'))
    crc.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=25, color='orange', weight=1,
                        popup=folium.Popup('Rayon 25m'))
    crc.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=100, color='orange', weight=1,
                        popup=folium.Popup('Rayon 100m'))
    crc.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=200, color='orange', weight=1,
                        popup=folium.Popup('Rayon 200m'))
    crc.add_to(mp)
    
    # La trace si disponible
    dfTracesForms.dropna(subset=['ID liste'], inplace=True)

    # a. Les lignes reliant les points
    if len(dfTracesForms) > 1:
        dfTracesForms['obseur_lon_sfd'] = dfTracesForms.obseur_lon.shift(-1)
        dfTracesForms['obseur_lat_sfd'] = dfTracesForms.obseur_lat.shift(-1)

        commListe = dfTracesForms.iloc[0]['Commentaire de la liste']
        compListe = dfTracesForms.iloc[0]['Liste complète ?']
        lines = list(zip(zip(dfTracesForms.obseur_lat, dfTracesForms.obseur_lon),
                         zip(dfTracesForms.obseur_lat_sfd, dfTracesForms.obseur_lon_sfd)))[:-1]
        pline = folium.PolyLine(lines, color='blue', weight=1, opacity=0.6,
                                popup=folium.Popup('#{} {} ({}complète)'.format(numPoint, commListe, '' if compListe else 'in')))
        pline.add_to(mp)

    # b. Les points
    if len(dfTracesForms) > 0:
        for _, sPt in dfTracesForms.iterrows():
            mrk = folium.CircleMarker(location=(sPt.obseur_lat, sPt.obseur_lon), 
                                      popup=folium.Popup('#{}: {}'.format(numPoint, sPt.NumPt)),
                                      radius=5, weight=2, color='red', fill=True)
            mrk.add_to(mp)

mp.fit_bounds(mp.get_bounds())

# Save map as shareable / web-publishable interactive one.
mp.save(f'tmp/ACDC2019b-Points{obserAbbv}-ComparaisonGeolocTraceEtMemoire.html')

# Display map.
mp

# C. Filtrage et cartographie des données et traces

In [ ]:
fn, obser

In [ ]:
dfObsSel = dfObs.copy()

In [ ]:
dfObsSel['Commentaire de la liste'].unique()

In [ ]:
dfObsSel.Commune.unique()

## 1. Filtrage

### a. Impluvium Volvic 2019 Romain

In [ ]:
obsSelName = 'impluvium'

In [ ]:
# Uniquement les EPOC sur Saint-Ours, Pulvérières, Charbonnières-les-Varennes
dfObsSel.drop(dfObsSel[dfObsSel['Commentaire de la liste'].isnull()].index, inplace=True)
dfObsSel.drop(dfObsSel[~dfObsSel.Commune.isin(['Saint-Ours', 'Pulvérières', 'Charbonnières-les-Varennes'])].index, inplace=True)

dfObsSel.drop(dfObsSel[~dfObsSel['Commentaire de la liste'].str.contains('epoc', case=False)].index, inplace=True)

len(dfObsSel)

In [ ]:
dfObsSel

### b. Forêt de Marcenat

In [ ]:
obsSelName = 'marcenat03'

In [ ]:
# Uniquement les EPOC sur Saint-Didier-la-Forêt, Saint-Rémy-en-Rollat (03)
dfObsSel.drop(dfObsSel[~dfObsSel.Commune.isin(['Saint-Didier-la-Forêt', 'Saint-Rémy-en-Rollat'])].index, inplace=True)

len(dfObsSel)

### c. Planèze

In [ ]:
obsSelName = 'planeze15'

In [ ]:
commPlaneze = ['Cussac', 'Neuvéglise', 'Ternes (Les)', 'Sériers',
               'Tanavelle', 'Valuéjols', 'Paulhac', 'Villedieu', 'Ussel',
               'Coltines', 'Talizat']

In [ ]:
# Uniquement les EPOC sur Saint-Didier-la-Forêt, Saint-Rémy-en-Rollat (03)
dfObsSel.drop(dfObsSel[~dfObsSel.Commune.isin(commPlaneze)].index, inplace=True)

len(dfObsSel)

## 2. Extraction et décodage des traces

In [ ]:
# On peut donc récupérer trace de chaque liste et en extraire les points individuels
formIndCols = ['ID liste', 'Date', 'Liste complète ?', 'Commentaire de la liste']
dfTraces = dfObsSel[formIndCols + ['Trace']].groupby(formIndCols).first()
dfTraces

In [ ]:
len(dfTraces)

In [ ]:
def decoderTrace(trace):
    return [[float(num) for num in xy.strip().split(' ')] for xy in trace[len('LINESTRING('):-len(')')].split(',')]
    
dfTraces.Trace = dfTraces.Trace.apply(decoderTrace)
dfTraces = dfTraces.Trace.apply(pd.Series).stack().reset_index()
dfTraces[['obseur_lon', 'obseur_lat']] = dfTraces.loc[:, 0].apply(pd.Series)
dfTraces.drop(columns=[0], inplace=True)
dfTraces.rename(columns=dict(level_4='NumPt'), inplace=True)
dfTraces

## 3. Cartographie

In [ ]:
dfObsSel.Date.min(), dfObsSel.Date.max()

In [ ]:
# Les formulaires à traiter et leurs nbres de données
dfObsSel[['ID liste', 'Trace']].groupby('ID liste').count().rename(columns=dict(Trace='NbObs'))

In [ ]:
dfObsSel.columns

In [ ]:
# Cartographie des formulaires
# La carte et les couches
mp = folium.Map()
folium.TileLayer(**mdSatArcGis).add_to(mp)

# Le contrôle pour changer de couche
folium.LayerControl().add_to(mp)

# Pour chaque formulaire sélectionné
for idListe in sorted(dfTraces['ID liste'].unique()):
    
    # Les données
    dfObsListe = dfObsSel[dfObsSel['ID liste'] == idListe]
    for indObs, sObs in dfObsListe.iterrows():
        mrk = folium.CircleMarker(location=(sObs['Lat (WGS84)'], sObs['Lon (WGS84)']), 
                                  radius=4, color='cyan', fill=True,
                                  popup=folium.Popup('#{} {} {} {} {} (code {})' \
                                                     .format(indObs, sObs.Date, sObs.Horaire,
                                                             sObs.Nombre, sObs['Nom latin'], sObs['Code atlas'])))
        mrk.add_to(mp)
    
    # La position observateur estimée, avec cerles concentriques r=10m + STOC EPS pour l'échelle
    #latMem, lonMem = dfTracesForms.iloc[0][['lat_mem', 'lon_mem']]
    #mrk = folium.CircleMarker(location=(latMem, lonMem), 
    #                          radius=6, color='orange', fill=True,
    #                          popup=folium.Popup('#{} Géoloc. de mémoire'.format(numPoint)))
    #mrk.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=10, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 10m'))
    #crc.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=25, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 25m'))
    #crc.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=100, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 100m'))
    #crc.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=200, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 200m'))
    #crc.add_to(mp)
    
    # La trace
    dfTracesForms = dfTraces[dfTraces['ID liste'] == idListe].copy()
    
    # a. Les lignes reliant les points
    if len(dfTracesForms) > 1:
        dfTracesForms['obseur_lon_sfd'] = dfTracesForms.obseur_lon.shift(-1)
        dfTracesForms['obseur_lat_sfd'] = dfTracesForms.obseur_lat.shift(-1)

        commListe = dfTracesForms.iloc[0]['Commentaire de la liste']
        lines = list(zip(zip(dfTracesForms.obseur_lat, dfTracesForms.obseur_lon),
                         zip(dfTracesForms.obseur_lat_sfd, dfTracesForms.obseur_lon_sfd)))[:-1]
        pline = folium.PolyLine(lines, color='blue', weight=2, opacity=0.6,
                                popup=folium.Popup('#{} {}'.format(idListe, commListe)))
        pline.add_to(mp)

    # b. Les points
    if len(dfTracesForms) > 0:
        for _, sPt in dfTracesForms.iterrows():
            mrk = folium.CircleMarker(location=(sPt.obseur_lat, sPt.obseur_lon), 
                                      popup=folium.Popup('#{}: {}'.format(idListe, sPt.NumPt)),
                                      radius=3, weight=2, color='red', fill=True)
            mrk.add_to(mp)

mp.fit_bounds(mp.get_bounds())

# Display map.
mp

In [ ]:
# Save map as shareable / web-publishable interactive one.
mFn = pl.Path(fn).with_suffix(f'.{obsSelName}.html')
mp.save(str(mFn))

HTML(f'<a href="{mFn}" target="_blank">{mFn}</a>')

# D. Cartographie des communes d'une sélection de données

In [ ]:
# Chargement d'un fichier des sites à peu près à jour
dfSites = pd.read_csv('tmp/SitesFA-20190522.csv', sep='\t', skiprows=1)
dfSites.head()

In [ ]:
dfCommunes.columns

In [ ]:
# Et voici en gros la mairie de chaque commune.
dfCommunes = dfSites[dfSites.Nom == dfSites.Commune.apply(lambda s: s + ' (bourg)')]
len(dfCommunes)

In [ ]:
# Filtrage
dfCommunesSel = dfCommunes[dfCommunes.Commune.isin(dfObsSel.Commune.unique())]

In [ ]:
# Ou pas filtrage
dfCommunesSel = dfCommunes

In [ ]:
# La carte et les couches
mp = folium.Map()
folium.TileLayer(**mdSatArcGis).add_to(mp)

# Le contrôle pour changer de couche
folium.LayerControl().add_to(mp)

# Pour chaque formulaire sélectionné
for _, sCom in dfCommunesSel.iterrows():
    
    # La position observateur estimée, avec cerles concentriques r=10m + STOC EPS pour l'échelle
    mrk = folium.CircleMarker(location=sCom[['Latitude (D.d)', 'Longitude (D.d)']], 
                              radius=10, color='fuchsia', fill=True,
                              popup=folium.Popup('{} - {:02d} ({}m)'.format(sCom.Nom, int(sCom.INSEE)//1000, sCom.Altitude)))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())

# Display map.
mp

# E. Tests du module visionature

In [ ]:
# Tests unitaires automatisés intégrés au module : python visionature.py -t

# F. Archives : Exploitation traces formulaires

Ancien code industrialisé dans module **visionat**

In [ ]:
# Projeteurs WGS84 et UTM31, et transformeur du 1er au 2nd.
KProjWgs84 = pyproj.Proj(init='epsg:4326') # WGS 84 : long, lat en degrés
KProjUtm31  = pyproj.Proj(init='epsg:32631') # WGS 84 - UTM 31N : long, lat en m

KTransformer = pyproj.Transformer.from_proj(KProjWgs84, KProjUtm31)

# Conversion WGS84 (long, lat en degrés) => UTM31 (x, y en m)
def wgs84toUtm(sLonLat):
    return pd.Series(KTransformer.transform(sLonLat[0], sLonLat[1]))

# Fonction de calcul de distance sphérique
#def sphericalDistance(aLon1, aLat1, aLon2, aLat2):
#
#    aX1, aY1 = KTransformer.transform(aLon1, aLat1)
#
#    aX2, aY2 = KTransformer.transform(aLon2, aLat2)
#
#    return np.hypot(aX2 - aX1, aY2 - aY1)

## I. Chargement des données (exports de Faune-France)

### 1. Paramètres d'import / filtrage (fichier, observateur, commentaire liste, ...)

In [ ]:
tracesManu = False

In [ ]:
# ACDC 2019 JPM (en fait, trace enregistrée uniquement sur le 2nd passage, et pour 15 points sur 17)
fn = 'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602ff.xlsx'
obser, obserAbbv = 'Jean-Philippe Meuret', 'JPM'

In [ ]:
# Formulaires Romain avec trace du 25/05/2019 au 25/01/2020 (tous)
# * Impluvium: "epoc" & Saint-Ours, Pulvérières, Charbonnière-les-Varennes
# * et le reste : des EPOC et des transects
fn = 'tmp/FormulairesRR-traces-20190525-20200125ff.xlsx'
comntRE = 'ACDC'
obser, obserAbbv = 'Romain Riols', 'RR'

In [ ]:
# ACDC 2019 Romain 25/05/2019
fn = 'tmp/ACDC2019-Naturalist-FormulairesRR20190425ff.xlsx'
comntRE = 'ACDC'
obser, obserAbbv = 'Romain Riols', 'RR'

In [ ]:
# Test JPM jardin 23/05/2019
#fn = f'tmp/NaturalistTestTrace-JardinJPM20190523ff.xlsx'
#comntRegexp = '???'
#obser, obserAbbv = 'Jean-Philippe Meuret', 'JPM'

In [ ]:
# Transects Vergers de Tallende Cyrille printemps 2020
fn = 'transects/ExportTransects2020T2-CJS-FF.xlsx'
comntRE = 'verger.*tallende'
obser, obserAbbv = 'Cyrille Jallageas', 'CJS'
tracesManu = False # Loaded from KML, not Naturalist traces

### 2. Chargement

In [ ]:
# Lecture du fichier
dfObs = pd.read_excel(fn)
len(dfObs)

In [ ]:
# Nettoyage colonnes inutiles
#dfObs.drop(columns=[col for col in dfObs.columns if col.startswith('SEARCH_EXPORT')], inplace=True)
#dfObs.columns

In [ ]:
# Aperçu
obsCols = ['ID liste', 'Liste complète ?', 'Commentaire de la liste',
           'Date', 'Horaire', 'Lieu-dit', 'Commune', 'Nom espèce',
           'Estimation', 'Nombre', 'Détails', 'Code atlas',
           'Lat (WGS84)', 'Lon (WGS84)', 'UTM X [m]', 'UTM Y [m]',
           'Remarque', 'Remarque privée', 'Trace']

dfObs = dfObs[obsCols]
dfObs

In [ ]:
dfObs['Commentaire de la liste'].nunique(), dfObs['Commentaire de la liste'].unique()

## II. Filtrage des données

* formulaires
* avec commentaire ad-hoc (si spécifié via comntRE)

In [ ]:
#df = dfObs.drop(dfObs[dfObs['Commentaire de la liste'].isnull()].index)
#df[~df['Commentaire de la liste'].str.contains(comntRE, case=False)]['Commentaire de la liste']

In [ ]:
# Uniquement les données des formulaires
dfObs.drop(dfObs[dfObs['ID liste'] == 0].index, inplace=True)
len(dfObs)

In [ ]:
# Uniquement les données respectant le critère de commentaire liste
if comntRE:
    dfObs.drop(dfObs[dfObs['Commentaire de la liste'].isnull()].index, inplace=True)
    dfObs.drop(dfObs[~dfObs['Commentaire de la liste'].str.contains(comntRE, case=False)].index,
               inplace=True)
len(dfObs)

In [ ]:
# Uniquement ceux avec trace si on ne les a pas par ailleurs
if not tracesManu:
    dfObs.drop(dfObs[dfObs['Trace'].isnull()].index, inplace=True)

len(dfObs[obsCols])

## III. Autres filtrages / traitements spécifiques des données

### 1. Transects de Cyrille dans les vergers de Tallende au printemps 2020

* filtrage du 1er transect pour en faire un transect nord décent
* ajout colonne "transect" (Nord, Sud1, Sud2)

In [ ]:
dfObs[(dfObs['Date'] == '2020-03-28') & (dfObs['Horaire'] > '09:40')]

In [ ]:
sLabels2Drop = dfObs[(dfObs['Date'] == '2020-03-28') & (dfObs['Horaire'] > '09:40')].index
dfObs.drop(sLabels2Drop, inplace=True)

len(dfObs)

In [ ]:
dfObs['Transect'] = \
    dfObs['Commentaire de la liste'].apply(lambda s: 'Sud' if s.lower().find('transect s') >= 0 else 'Nord')

In [ ]:
(dfObs[['ID liste', 'Transect']].drop_duplicates() == vnds.dfData[['ID liste', 'Transect']].drop_duplicates()).all().all()

## IV. Examen des données

In [ ]:
# Toutes des listes complètes ?
dfObs[dfObs['Liste complète ?'] != 1]

In [ ]:
# Vérifier que la trace de chaque liste est présente à l'identique dans toutes les données de la liste
assert all(dfObs[['ID liste', 'Trace']].groupby('ID liste').nunique().Trace == 1)

In [ ]:
# Les formulaires à traiter
dfObs[['ID liste', 'Liste complète ?', 'Commentaire de la liste', 'Trace']].groupby('ID liste').first()

In [ ]:
# Les formulaires à traiter : nbres de données
dfObs[['ID liste', 'Date', 'Trace']].groupby(['Date', 'ID liste']).count().Trace

In [ ]:
# Nbre de données transect sud
dfObs.loc[dfObs['Commentaire de la liste'].str.contains('transect s', case=False)].groupby(['Date', 'ID liste']) \
     .count().Trace

In [ ]:
# Moyenne
dfObs.loc[dfObs['Commentaire de la liste'].str.contains('transect s', case=False)].groupby(['Date', 'ID liste']) \
     .count().Trace.mean()

In [ ]:
# Nbre de données transect nord
dfObs.loc[dfObs['Commentaire de la liste'].str.contains('transect n', case=False)].groupby(['Date', 'ID liste']) \
     .count().Trace

In [ ]:
# Moyenne
dfObs.loc[dfObs['Commentaire de la liste'].str.contains('transect n', case=False)].groupby(['Date', 'ID liste']) \
     .count().Trace.mean()

## V. Extraction et décodage des traces GPS

In [ ]:
# On peut donc récupérer la trace de chaque liste et en extraire les points individuels
formIndCols = ['ID liste', 'Date', 'Liste complète ?', 'Commentaire de la liste']
dfForms = dfObs[formIndCols + ['Trace']].groupby(formIndCols).first()
dfForms

In [ ]:
# Exemple de trace
dfForms.iloc[0].Trace

In [ ]:
def decoderTrace(trace):
    if pd.isnull(trace):
        return []
    else:
        return [[float(num) for num in xy.strip().split(' ')] \
                for xy in trace[len('LINESTRING('):-len(')')].split(',')]
    
dfTracesGps = dfForms.copy()

dfTracesGps.Trace = dfTracesGps.Trace.apply(decoderTrace)
dfTracesGps = dfTracesGps.Trace.apply(pd.Series).stack().reset_index()
dfTracesGps[['obseur_lon', 'obseur_lat']] = dfTracesGps.loc[:, 0].apply(pd.Series)
dfTracesGps.drop(columns=[0], inplace=True)
dfTracesGps.rename(columns=dict(level_4='NumPt'), inplace=True)
dfTracesGps

In [ ]:
# Calcul des coordonnées UTM 31 (métriques) de la trace.
dfTracesGps[['obseur_lon_utm', 'obseur_lat_utm']] = \
    dfTracesGps[['obseur_lon', 'obseur_lat']].apply(wgs84toUtm, axis='columns')

In [ ]:
dfTracesGps

## VI. Filtrages spécifiques des traces GPS

### 1. Transects de Cyrille dans les vergers de Tallende au printemps 2020

Couper le 1er transect pour qu'il colle bien avec les transects nord des jours suivants

In [ ]:
dfTracesGps[(dfTracesGps.Date == '2020-03-28') & ~(dfTracesGps.NumPt.between(8, 170))]

In [ ]:
sLabels2Drop = dfTracesGps[(dfTracesGps.Date == '2020-03-28') & ~(dfTracesGps.NumPt.between(8, 171))].index
dfTracesGps.drop(sLabels2Drop, inplace=True)

In [ ]:
dfTracesGps.head()

## VII. Chargement des vrais parcours des transects

(tracés précisément à la main via GeoPortail ... pas de pb avec l'imprécision du GPS)

In [ ]:
kmlNameSpaces = \
  { 'gx' : 'http://www.google.com/kml/ext/2.2',
    'kml' : 'http://www.opengis.net/kml/2.2',
    'atom' : 'http://www.w3.org/2005/Atom' }

### 1. Transects de Cyrille dans les vergers de Tallende au printemps 2020

In [ ]:
# Transect nord
if tracesManu:

    kmlRoot = etree.ElementTree().parse('transects/TransectNord-VergersTallende-CJS-2020T2.kml')

    plMark = kmlRoot.find('kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
    transLine = plMark.find('kml:LineString/kml:coordinates',
                           namespaces=kmlNameSpaces).text.strip()
    dfTransN = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in transLine.split(' ')],
                            columns=['obseur_lon', 'obseur_lat'])
    dfTransN = dfTransN.reset_index().rename(columns=dict(index='NumPt'))

    print(len(dfTransN), dfTransN.columns)

In [ ]:
# Transect sud
if tracesManu:
    
    kmlRoot = etree.ElementTree().parse('transects/TransectSud-VergersTallende-CJS-2020T2.kml')

    plMark = kmlRoot.find('kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
    transLine = plMark.find('kml:LineString/kml:coordinates',
                           namespaces=kmlNameSpaces).text.strip()
    dfTransS = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in transLine.split(' ')],
                            columns=['obseur_lon', 'obseur_lat'])
    dfTransS = dfTransS.reset_index().rename(columns=dict(index='NumPt'))
    
    print(len(dfTransS), dfTransS.columns)

In [ ]:
if tracesManu:
    
    dfTransN['Transect'] = 'Nord'
    dfTransS['Transect'] = 'Sud'
    dfTracesManu = dfTransN.append(dfTransS, ignore_index=True)

    print(dfTracesManu.head())

In [ ]:
# Calcul des coordonnées UTM 31 (métriques) des traces.
if tracesManu:
    
    dfTracesManu[['obseur_lon_utm', 'obseur_lat_utm']] = \
        dfTracesManu[['obseur_lon', 'obseur_lat']].apply(wgs84toUtm, axis='columns')
    
    print(dfTracesManu.head())

## VIII. Tracé cartographique des données de formulaires et de leur trace

In [ ]:
print(', '.join("'{}'".format(date.date()) for date in dfObs.Date.drop_duplicates()))

### 1. Sélection des listes à tracer en même temps

In [ ]:
# Soit en dur.
idsListe = [876746] #[955938, 956038] #[880581, 880582] #[876746]

In [ ]:
dfObs.groupby('Date').apply(lambda df: list(df['ID liste'].unique()))

In [ ]:
# Soit: Iteration sur les jours : liste des IDs de formulaires chaque jour
itDates = iter(dfObs.groupby('Date').apply(lambda df: list(df['ID liste'].unique())))

In [ ]:
idsListe = next(itDates)
idsListe

### 2. Extraction des points des traces du jour et des données

In [ ]:
if tracesManu:
    dfTracesForms = pd.DataFrame()
    for idListe in idsListe:
        comntListe = dfTracesGps[dfTracesGps['ID liste'] == idListe].iloc[0]['Commentaire de la liste']
        transect = 'Sud' if comntListe.lower().find('transect s') >= 0 else 'Nord'
        dfTraceManu = dfTracesManu[dfTracesManu.Transect == transect].copy()
        dfTraceManu['ID liste'] = idListe
        dfTraceManu['Commentaire de la liste'] = comntListe
        dfTracesForms = dfTracesForms.append(dfTraceManu, ignore_index=True,)
else:
    dfTracesForms = dfTracesGps.loc[dfTraces['ID liste'].isin(idsListe)].copy()

len(dfTracesForms)

In [ ]:
dfObsListe = dfObs.loc[dfObs['ID liste'].isin(idsListe),
                       ['Lon (WGS84)', 'Lat (WGS84)', 'Date', 'Horaire', 'Nom espèce', 'Nombre', 'Code atlas']]
len(dfObsListe)

In [ ]:
dfTracesForms

### 3. Tracé

In [ ]:
showOnMap(dfTracesForms.rename(columns={'ID liste': 'formId', 'NumPt': 'ptNum',
                                      'obseur_lon': 'lon', 'obseur_lat': 'lat'}),
          dfObsListe.rename(columns={'Lon (WGS84)': 'lon', 'Lat (WGS84)': 'lat' }), mapSrc=mdOTM,
          trTitle='#{} {}'.format(','.format(idsListe), dfTracesForms.iloc[0]['Commentaire de la liste']))

## IX. Calcul des distances observateur - oiseau

(la distance minimale entre l'oiseau et la poly-ligne du transect)

In [ ]:
dfObs.columns

In [ ]:
DTransPolyLines = dict()
def transectPolyLine(sObs):

    if tracesManu:
        
        transect = sObs['Transect']
        if transect not in DTransPolyLines:
            dfTransTrace = dfTracesManu.loc[dfTracesManu.Transect == transect, ['obseur_lon_utm', 'obseur_lat_utm']]
            DTransPolyLines[transect] = \
                geometry.LineString([(x, y) for x, y in dfTransTrace.itertuples(index=False)])
            
    else:

        transect = sObs['ID liste']
        if idListe not in DTransPolyLines:
            dfTransTrace = dfTracesGps.loc[dfTracesGps['ID liste'] == transect, ['obseur_lon_utm', 'obseur_lat_utm']]
            DTransPolyLines[transect] = \
                geometry.LineString([(x, y) for x, y in dfTransTrace.itertuples(index=False)])
        
    return DTransPolyLines[transect]

def distance2Transect(sObs):
    
    return geometry.Point(sObs['UTM X [m]'], sObs['UTM Y [m]']).distance(transectPolyLine(sObs))
    
dfObs['Distance'] = \
    dfObs[['ID liste', 'Transect', 'UTM X [m]', 'UTM Y [m]',]].apply(distance2Transect, axis='columns')

dfObs

In [ ]:
dfObs[dfObs['ID liste'] == idListe].Distance.hist()

In [ ]:
dfObs.Distance.hist(bins=40)

# G. Archives : Essai extraction traces via format JSON / XML

(avant de savoir qu'elles n'y sont sont pas, le 25/01/2020)

In [ ]:
#src = 'fa'
src = 'ff'

In [ ]:
fn = f'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602{src}.json'

In [ ]:
fn = f'tmp/NaturalistTestTrace-JardinJPM20190523{src}.json'

In [ ]:
fn = 'tmp/NaturalistTestTrace-FormRomainsAutHiv201920fa.json'

In [ ]:
dObsTr = json.load(open(fn))
type(dObsTr), type(dObsTr['data']), dObsTr['data'].keys(), type(dObsTr['data']['forms']), type(dObsTr['data']['sightings'])

In [ ]:
dict(nbFormulaires=len(dObsTr['data']['forms']), nbObsHorsFormulaires=len(dObsTr['data']['sightings']))

In [ ]:
#dObsTr

In [ ]:
#dObsTr['data']['forms'][8]

In [ ]:
for dForm in dObsTr['data']['forms']:
    print(dForm['@id'], ':', dForm['time_start'], dForm['time_stop'], dForm['lat'], dForm['lon'],
                             dForm.get('comment', ''), '=>', len(dForm['sightings']))

In [ ]:
def flattenForm(form):
    flat = odict()
    for k, v in form.items():
        if k == 'sightings':
            continue
        if isinstance(v, dict):
            for sk, sv in v.items():
                if k != 'protocol' or sk == 'protocol_name':
                    flat.update(**{ 'form_'+k+'_'+sk: sv})
        else:
            flat.update(**{ 'form_'+k: v})
    return flat

dfForms = pd.DataFrame(data=[flattenForm(form) for form in dObsTr['data']['forms']])
dfForms.set_index('form_@id', inplace=True)
dfForms

In [ ]:
def flattenSight(sight, formId):
    flat = odict([('form_@id', formId)])
    for k, v in sight.items():
        if isinstance(v, list):
            v = v[0]
        for sk, sv in v.items():
            if isinstance(sv, dict):
                for ssk, ssv in sv.items():
                    flat.update(**{ k+'_'+sk+'_'+ssk: ssv})
            else:
                flat.update(**{ k+'_'+sk: sv})
    return flat

ldfSights = list()
for form in dObsTr['data']['forms']:
    dfSights = pd.DataFrame(data=[flattenSight(sight, form['@id']) for sight in form['sightings']])
    dfSights = dfSights.join(dfForms, on=['form_@id'])
    ldfSights.append(dfSights)
    
dfSights = pd.concat(ldfSights, sort=False, ignore_index=True)
dfSights

In [ ]:
dfSights.columns

In [ ]:
if 'form_protocol_protocol_name' in dfSights.columns:
    dfSights.drop(dfSights[dfSights['form_protocol_protocol_name'] == 'STOC_EPS'].index, inplace=True)
dfSights['form_comment'].fillna('', inplace=True)
dfSights.drop(dfSights[~dfSights['form_comment'].str.contains('ACDC', case=False)].index, inplace=True)

dfSights

In [ ]:
dfSights['date_@ISO8601'].min(), dfSights['date_@ISO8601'].max()

In [ ]:
dfSights[['form_@id']+[col for col in dfSights.columns if col.endswith('lat') or col.endswith('lon')]].head(30)

In [ ]:
df = pd.DataFrame([dict(a=1, b=2), dict(a=3, b=4), dict(a=5, b=6)], index=[1, 2, 3])
df

In [ ]:
dIdList2Trans = { 876746: 'Nord',
                  880581: 'Nord', 880582: 'Sud', 888657: 'Nord', 888658: 'Sud',
                  893072: 'Nord', 893073: 'Sud', 907198: 'Nord', 907199: 'Sud',
                  926044: 'Nord', 926208: 'Sud', 938061: 'Nord', 938221: 'Sud',
                  945756: 'Nord', 946015: 'Sud', 955938: 'Nord', 956038: 'Sud',
                  964083: 'Nord', 964213: 'Sud', 999946: 'Nord', 999947: 'Sud' }
if not tracesManu:
    del dIdList2Trans[926044]

dIdList2Trans

In [ ]:
# a. Chargement des fichiers KML et passage des DataFrame ad-hoc au module
print('Setting new GPS traces (one trace table per list) ...')
ddfTransects = dict()
for trans in set(dIdList2Trans.values()):
    kmlRoot = etree.ElementTree().parse(f'transects/Transect{trans}-VergersTallende-CJS-2020T2.kml')
    plMark = kmlRoot.find('kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
    transLine = plMark.find('kml:LineString/kml:coordinates',
                           namespaces=kmlNameSpaces).text.strip()
    dfTrans = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in transLine.split(' ')],
                           columns=['lon', 'lat'])
    dfTrans = dfTrans.reset_index().rename(columns=dict(index='ptIndx'))
    print('Transect "{}": {} points'.format(trans, len(dfTrans)))
    ddfTransects[trans] = dfTrans

dTraces = { lstId: ddfTransects[trans] for lstId, trans in dIdList2Trans.items() }

vnds.setListTraces(dTraces)

dfTracesGps = vnds.listTraces()

dfTracesGps

In [ ]:
# b. Soin au module de charger lui-même les fichiers
print('Setting new GPS traces (one trace KML file per list) ...')
dTraces = { lstId: f'transects/Transect{transect}-VergersTallende-CJS-2020T2.kml' \
            for lstId, transect in dIdList2Trans.items() }

vnds.setListTraces(dTraces)

dfTracesGps2 = vnds.listTraces()

dfTracesGps2


In [ ]:
dfCodesAtlas = pd.read_csv('visionat/VisioNatureCodesAtlas.txt', sep='\t',
                            index_col=0, usecols=['Codes Biolovision', 'Texte FR'])
dfCodesAtlas.index

In [ ]:
s = "3.1300199031829834,45.66334557709624 3.1302559375762935,45.66330808661448 3.130652904510498,45.66317312067224 3.1309103965759277,45.66306814693661 3.1310713291168213,45.663030656269086 3.13135027885437,45.66300066371696 3.1315112113952637,45.6629781692923 3.1320691108703613,45.662618257268974 3.1323266029357915,45.66252827890162 3.133056163787842,45.66231832881542 3.1333351135253906,45.66222834996603 3.133699893951416,45.66215336748104 3.1348907947540283,45.662070886631625 3.1362748146057133,45.66198840566065 3.1373798847198486,45.66195091427005 3.1380558013916016,45.661905924568174 3.138549327850342,45.66192092113948 3.1392788887023926,45.66198090738453 3.1398689746856694,45.66206338836653"
' '.join(reversed(s.split()))